In [1]:
import requests

In [2]:
from pydantic import BaseModel, ConfigDict, Field, field_validator, IPvAnyAddress

In [13]:
class IPGeo(BaseModel):
    model_config = ConfigDict(
        extra="ignore"
    )
    
    ip: IPvAnyAddress
    country: str | None = None
    country_code: str | None = Field(default=None, min_length=2, max_length=2)
    country_code3: str | None = Field(default=None, min_length=3, max_length=3)
    city: str | None = None
    region: str | None = None
    timezone: str | None = None
    organization_name: str | None = None
    
    @field_validator("organization_name", mode="after")
    @classmethod
    def set_unknown_to_none(cls, value: str) -> str | None:
        if value.casefold() == "unknown":
            return None
        return value

In [14]:
IPGeo(ip="8.8.8.8", country="test", country_code3="USA", organization_name="UnKnoWn")

IPGeo(ip=IPv4Address('8.8.8.8'), country='test', country_code=None, country_code3='USA', city=None, region=None, timezone=None, organization_name=None)

In [41]:
url_query="https://get.geojs.io/v1/ip/geo/{ip_address}.json"

In [42]:
url = url_query.format(ip_address="8.8.8.8")

In [43]:
response = requests.get(url)

In [45]:
response.raise_for_status()

In [46]:
response_json = response.json()

In [49]:
response_json

{'area_code': '0',
 'country_code': 'US',
 'country_code3': 'USA',
 'continent_code': 'NA',
 'asn': 15169,
 'latitude': '37.751',
 'longitude': '-97.822',
 'ip': '8.8.8.8',
 'organization_name': 'GOOGLE',
 'organization': 'AS15169 GOOGLE',
 'timezone': 'America/Chicago',
 'accuracy': 1000,
 'country': 'United States'}

In [51]:
data = IPGeo.model_validate(response_json)

In [52]:
data

IPGeo(ip=IPv4Address('8.8.8.8'), country='United States', country_code='US', country_code3='USA', city=None, region=None, timezone='America/Chicago', organization_name='GOOGLE')

In [54]:
url = url_query.format(ip_address="23.62.177.155")
response = requests.get(url)
response_json = response.json()
IPGeo.model_validate(response_json)

IPGeo(ip=IPv4Address('23.62.177.155'), country='United States', country_code='US', country_code3='USA', city='El Segundo', region='California', timezone='America/Los_Angeles', organization_name='AKAMAI-AS')

In [59]:
url="https://get.geojs.io/v1/ip/geo.json"
response = requests.get(url)
response_json = response.json()
data = IPGeo.model_validate(response_json)
data

IPGeo(ip=IPv6Address('2a01:110f:480a:de00:c2fc:d107:8511:262d'), country='Poland', country_code='PL', country_code3='POL', city='Warsaw', region='Mazovia', timezone='Europe/Warsaw', organization_name='Orange Polska Spolka Akcyjna')

In [60]:
data.ip

IPv6Address('2a01:110f:480a:de00:c2fc:d107:8511:262d')